In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.style.use('ggplot')

In [ ]:
##IMPORTED LIBRARIES ->READING DATA

In [89]:
df = pd.read_csv("quikr_car.csv")

In [ ]:
##VIEWING DATA

In [90]:
df.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [91]:
df.shape

(892, 6)

In [92]:
##QUALITY Problems -> 
##checking each column by df['col_name'].unique()

In [93]:
##year has many garbage 
##values which are non-year

In [94]:
##Price has garbage values as well 
##convert to int and remove commas

##kms_driven should have int it has strings with 
##commas and kms at end aswell

In [95]:
##fuel_type has nan values

In [96]:
##Name is very inconsistent so we keep the 
##first three words only

In [97]:
backup = df.copy()

In [98]:
# DATA CLEANING

In [99]:
df = df[df['year'].str.isnumeric()]

In [100]:
df['year'] = df['year'].astype(int)

In [101]:
df=df[df['Price']!='Ask For Price']
df['Price']=df['Price'].str.replace(',','').astype(int)

In [102]:
df['kms_driven']=df['kms_driven'].str.split().str.get(0).str.replace(',','')

In [103]:
df=df[df['kms_driven'].str.isnumeric()]

In [104]:
df['kms_driven']=df['kms_driven'].astype(int)

In [105]:
df=df[~df['fuel_type'].isna()]

In [106]:
df['name']=df['name'].str.split().str.slice(start=0,stop=3).str.join(' ')

In [107]:
df=df.reset_index(drop=True)

In [108]:
df=df[df['Price']<6000000]

In [109]:
df.to_csv('Cleaned_Car_data.csv')

In [110]:
##SPLITTING THE DATA

In [111]:
X=df[['name','company','year','kms_driven','fuel_type']]
y=df['Price']

In [112]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [113]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [136]:

##One-hot encoding is often used in linear regression 
##(and other machine learning algorithms) when dealing 
##with categorical variables. Linear regression works well 
##with numerical data, but it struggles with categorical 
##variables because it assumes a linear relationship between 
##the input features and the output variable."""

"""For example, if you have a categorical variable 
"color" with three categories: red, green, and blue, 
one-hot encoding would represent these categories as 
three binary features: is_red, is_green, and is_blue. 
Each feature would take the value of 0 or 1, indicating 
the absence or presence of that category for a particular 
data point."""

'For example, if you have a categorical variable \n"color" with three categories: red, green, and blue, \none-hot encoding would represent these categories as \nthree binary features: is_red, is_green, and is_blue. \nEach feature would take the value of 0 or 1, indicating \nthe absence or presence of that category for a particular \ndata point.'

In [114]:
ohe=OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [115]:
ohe=OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])

OneHotEncoder()

In [116]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']),
                                    remainder='passthrough')

In [117]:
lr=LinearRegression()

In [118]:
pipe=make_pipeline(column_trans,lr)

In [119]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol'], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [120]:
y_pred=pipe.predict(X_test)

In [121]:
r2_score(y_test,y_pred)

0.5744075499462205

In [122]:
scores=[]
for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [123]:
np.argmax(scores)

302

In [124]:
scores[np.argmax(scores)]

0.8991138463319752

In [125]:
pipe.predict(pd.DataFrame(columns=X_test.columns,data=np.array(['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']).reshape(1,5)))

array([430287.74002343])

In [126]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)

0.8991138463319752

In [127]:
import pickle

In [128]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [129]:
pipe.predict(pd.DataFrame(columns=['name','company','year','kms_driven','fuel_type'],data=np.array(['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']).reshape(1,5)))

array([456740.35763086])

In [130]:
pipe.steps[0][1].transformers[0][1].categories[0]

array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat Diesel',
       'Chevrolet Beat LS', 'Chevrolet Beat LT', 'Chevrolet Beat PS',
       'Chevrolet Cruze LTZ', 'Chevrolet Enjoy', 'Chevrolet Enjoy 1.4',
       'Chevrolet Sail 1.2', 'Chevrolet Sail UVA', 'Chevrolet Spark',
       'Chevrolet Spark 1.0', 'Chevrolet Spark LS', 'Chevrolet Spark LT',
       'Chevrolet Tavera LS', 'Chevrolet Tavera Neo', 'Datsun GO T',
       'Datsun Go Plus', 'Datsun Redi GO', 'Fiat Linea Emotion',
       'Fiat Petra ELX', 'Fiat Punto Emotion', 'Force Motors Force',
       'Force Motors One', 'Ford EcoSport', 'Ford EcoSport Ambiente',
       'Ford EcoSport Titanium', 'Ford EcoSport Trend',
       'Ford Endeavor 4x4', 'Ford Fiesta', 'Ford Fiesta SXi', 'Ford Figo',
       'Ford Figo Diese